# Solución de problemas con regresión Lasso

En este tutorial conocerás cómo crear un modelo de regresión con la técnica de regularización con norma L1, también llamada regresión Lasso. Primero entrenaremos un modelo base y, posteriormente, realizaremos un ajuste de hiperparámetros mediante la técnica de validación cruzada con k-Folds, evaluando el mejor modelo obtenido. Adicionalmente, veremos cómo realizar un modelo de selección de variables, una característica particular de este tipo de regresión. En ese sentido, veremos cómo realizar los siguientes procesos:

1. Importar las librerías necesarias.
2. Cargar un conjunto de datos.
3. Preparar los datos para el modelado.
4. Entrenar un modelo de referencia usando regresión lineal.
5. Realizar una búsqueda de hiperparámetros.
6. Entrenar un modelo de selección de variables.

Utilizaremos el conjunto de datos correspondiente a la caracterización de casas y su precio. Nuestro objetivo es, entonces, obtener un modelo para predecir el precio de una vivienda dadas sus características.

## 1. Importación de librerías requeridas

Importaremos la librería `pandas` y `scikit-learn`. En particular, usaremos las siguientes clases para entrenar los modelos de regresión regularizada:

* `GridSearchCV`: clase para entrenar múltiples modelos variando sus parámetros. Se utiliza para hacer una búsqueda exhaustiva de los mejores valores para el entrenamiento de un modelo.
* `KFold`: clase para definir múltiples conjuntos de entrenamiento y validación sobre nuestro conjunto de datos.
* `MinMaxScaler`: clase para escalar los valores de nuestro conjunto de datos.
* `Lasso`: clase para crear y entrenar un modelo de regresión regularizada con norma L1.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## 2. Carga de datos

Realizaremos la carga de datos usando la función de Pandas `read_csv()`, especificando la ruta y el separador del archivo:

In [2]:
data_raw = pd.read_csv('data/kc_house_data.csv', sep=',')

Veremos los primeros datos del conjunto usando `head()`:

In [3]:
data_raw.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 3. Preparación de los datos

Definiremos la variable `data` para almacenar un conjunto de datos modificado. En ese sentido, podremos tener una copia de los datos originales almacenada en la variable `data_raw`, por si en algún momento es necesario recuperar información que haya sido modificada:

In [4]:
data = data_raw.copy()

### Eliminación de variables poco relevantes

Eliminaremos tres variables con poca relevancia para el precio de las viviendas: `id`, `date` y `zipcode`, haciendo uso de la función `drop()` sobre nuestro DataFrame:

In [5]:
data = data.drop(['id','date','zipcode'], axis=1)

Veremos el resultado con `data.head()`:

In [6]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


### División de datos

Ahora dividiremos el conjunto de datos resultante en un conjunto de entrenamiento y uno de pruebas mediante la función `train_test_split()`. Usaremos el 80% de los datos para el entrenamiento y el 20% restante para las pruebas:

In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=9)
train.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
17690,1364000.0,4,2.50,3560,8960,2.0,0,0,3,10,3560,0,2001,0,47.6903,-122.213,1660,7680
7711,815000.0,3,2.50,2415,2186,2.0,0,1,3,9,2415,0,1981,0,47.6506,-122.202,2660,2165
7878,205000.0,4,1.00,1030,6621,1.0,0,0,4,6,1030,0,1955,0,47.4857,-122.221,1420,6631
5409,439950.0,4,2.25,2780,15075,2.0,0,0,3,7,2780,0,1985,0,47.4770,-122.116,1650,25542
10047,362362.0,2,1.00,710,4000,1.0,0,0,3,6,710,0,1909,0,47.5535,-122.269,960,4000


Como los algoritmos supervisados implementados en `scikit-learn` necesitan que las variables de entrada estén separadas de la variable objetivo, usaremos la función `drop` y definiremos `x_train` y `y_train`, que representan los valores de las variables independientes y los valores de la variable objetivo, respectivamente:

In [8]:
x_train = train.drop(['price'],axis=1)
y_train = train['price']

### Estandarización

Para el caso de regresión regularizada, la escala en la que se encuentran las variables se vuelve relevante, a diferencia de la regresión lineal simple. Específicamente, dado que los métodos de regularización actúan sobre la magnitud de los coeficientes del modelo, todos deben estar en la misma escala. Es por eso que utilizaremos un objeto de la clase `MinMaxScaler()` que, por defecto, escala los valores de cada variable al rango [0,1]: 

In [9]:
scaler = MinMaxScaler()

Este objeto tiene el método `fit_transform()` para realizar la estandarización, pero retorna un arreglo en vez de un DataFrame. Por lo tanto, primero almacenaremos la información de las columnas en la variable `columns` y, finalmente, reconstruiremos el DataFrame:

In [10]:
columns = x_train.columns
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)

Visualizaremos los nuevos valores del conjunto de entrenamiento usando `head()`:

In [11]:
x_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,0.121212,0.31250,0.242217,0.005113,0.4,0.0,0.00,0.50,0.7,0.352876,0.0,0.878261,0.0,0.859579,0.254153,0.220802,0.008074
1,0.090909,0.31250,0.155277,0.001009,0.4,0.0,0.25,0.50,0.6,0.226217,0.0,0.704348,0.0,0.795721,0.263289,0.395903,0.001739
2,0.121212,0.12500,0.050114,0.003696,0.0,0.0,0.00,0.75,0.3,0.073009,0.0,0.478261,0.0,0.530481,0.247508,0.178778,0.006869
3,0.121212,0.28125,0.182992,0.008817,0.4,0.0,0.00,0.50,0.4,0.266593,0.0,0.739130,0.0,0.516487,0.334718,0.219051,0.028592
4,0.060606,0.12500,0.025816,0.002108,0.0,0.0,0.00,0.50,0.3,0.037611,0.0,0.078261,0.0,0.639537,0.207641,0.098231,0.003847


## 4. Entrenamiento de un modelo de referencia

Con el conjunto de datos modificado, empezaremos con el entrenamiento de un modelo de referencia, que nos permitirá ver cómo es el desempeño de un modelo de regresión lineal simple sobre este conjunto de datos. Definiremos un objeto de tipo `LinearRegression()` y lo entrenaremos con la función `fit()`, utilizando el conjunto de entrenamiento separado en las variables independientes `x_train` y la variable objetivo `y_train`:

In [12]:
reg_lineal = LinearRegression().fit(x_train, y_train)

Ahora veremos los coeficientes y el intercepto resultantes:

In [13]:
print ('Coeficientes: ', reg_lineal.coef_)
print ('Intercepto: ', reg_lineal.intercept_)

Coeficientes:  [-1045667.82788799   349811.79988409   800497.55201274   242049.55919058
     4639.74845105   584669.85722034   201276.17419331   127355.53285441
   961197.84442804  1020275.34410272   319434.29765598  -289152.79106719
    46639.14802456   348546.72281543  -144037.33147272   184630.22563774
  -362614.18447603]
Intercepto:  -335986.24078267557


Podemos ver qué coeficiente le corresponde a cada variable con el método `zip()`, usando los nombres de las variables almacenados en `x_train.columns`:

In [14]:
list(zip(x_train.columns, reg_lineal.coef_))

[('bedrooms', -1045667.8278879866),
 ('bathrooms', 349811.79988409486),
 ('sqft_living', 800497.5520127411),
 ('sqft_lot', 242049.5591905795),
 ('floors', 4639.748451048901),
 ('waterfront', 584669.8572203416),
 ('view', 201276.17419331183),
 ('condition', 127355.53285440998),
 ('grade', 961197.844428043),
 ('sqft_above', 1020275.3441027206),
 ('sqft_basement', 319434.2976559765),
 ('yr_built', -289152.79106718674),
 ('yr_renovated', 46639.14802455908),
 ('lat', 348546.7228154286),
 ('long', -144037.33147272174),
 ('sqft_living15', 184630.22563774252),
 ('sqft_lot15', -362614.18447602564)]

### Evaluación del modelo

Ahora utilizaremos el conjunto de pruebas para evaluar el desempeño del modelo. Separaremos las variables independientes y la variable objetivo, de la misma forma que para el conjunto de entrenamiento:

In [15]:
x_test = test.drop(['price'],axis=1)
y_test = test['price']

También usaremos la variable `scaler` para escalar las variables independientes del conjunto de pruebas. Ten en cuenta que solo utilizaremos la información de las variables disponible en el conjunto de entrenamiento, por lo que haremos uso del método `transform()`:

In [16]:
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Finalmente, realizaremos las predicciones. Utilizaremos tres métricas para evaluar el desempeño del modelo de referencia: la raíz del error cuadrático medio, el error absoluto medio y el coeficiente de determinación: 

In [17]:
y_pred = reg_lineal.predict(x_test)

print('------ Modelo de regresión lineal simple----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión lineal simple----
RMSE: 200731.58
MAE: 126120.34
R²: 0.71


Como puedes observar, el modelo de regresión simple se ajusta un 71% a los datos, aunque se tienen errores medios en el orden de los cientos de miles de dólares.

## 5. Búsqueda de hiperparámetros

Cambiar el valor de los hiperparámetros tiene un impacto directo sobre el desempeño del modelo resultante, por lo que queremos encontrar un valor que resulte en el mejor desempeño posible. A continuación, ejecutaremos un procedimiento de búsqueda exhaustiva, es decir, vamos a entrenar modelos con varios valores para `alpha` y, al finalizar, nos quedaremos con el valor del hiperparámetro que resulte en el modelo con el mejor desempeño.

Lo primero que haremos es definir un objeto de tipo `Lasso()`, que puede recibir un valor del hiperparámetro `alpha`. En este caso, al cambiar su valor durante la búsqueda, no es necesario especificarlo. Sin embargo, vamos a definir el parámetro `max_iter` para aumentar el número de iteraciones y, de esa forma, ayudar a la convergencia del modelo:

In [18]:
lasso = Lasso(max_iter=2000)

Ahora utilizaremos un diccionario para definir nuestro espacio de búsqueda de hiperparámetros, es decir, los valores que vamos a probar y sobre los que decidiremos cuál escoger. Almacenaremos estos valores en la variable `param_grid`:

In [19]:
param_grid = {'alpha': [1, 2, 5, 10, 15, 20, 100, 500]}

Además vamos a definir un objeto de la clase `KFold()`, que nos será útil para obtener una estimación del desempeño del modelo más realista que simplemente utilizando el conjunto de pruebas. La validación cruzada k-fold es un método que toma el conjunto de entrenamiento original y lo separa en k grupos, usando uno como validación y el resto (k-1) como entrenamiento. Después de definir los grupos, entrena el modelo con el conjunto de entrenamiento y lo evalúa con el conjunto de validación, repitiendo este proceso para cada uno de los k grupos. En este caso, definiremos `k=10` y usaremos el parámetro `shuffle=True` para indicar que se cambie el orden de los datos antes de separarlos en los grupos:

In [20]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

A continuación, vamos a utilizar `GridSearchCV` para realizar la búsqueda exhaustiva del mejor hiperparámetro. Definiremos el algoritmo `lasso`, los valores del hiperparámetro `param_grid` y la estrategia de validación cruzada `kfold`: 

In [21]:
modelos_grid = GridSearchCV(lasso, param_grid, cv=kfold, n_jobs=-1)

Finalmente, entrenaremos los modelos con los conjuntos definidos previamente (**Nota:** el entrenamiento puede tardar algunos minutos):

In [22]:
modelos_grid.fit(x_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=2000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1, 2, 5, 10, 15, 20, 100, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

Obtendremos el mejor parámetro usando el atributo `best_params_`:

In [23]:
print("Mejor parámetro: ", modelos_grid.best_params_)

Mejor parámetro:  {'alpha': 5}


Como puedes ver, el mejor valor del hiperparámetro `alpha` es 5. Podemos obtener el mejor modelo haciendo uso del atributo `best_estimator_`, y obtendremos sus coeficientes usando el atributo `coef_`:

In [24]:
mejor_modelo = modelos_grid.best_estimator_
list(zip(x_train.columns, mejor_modelo.coef_))

[('bedrooms', -1032700.9452536621),
 ('bathrooms', 348791.8782859398),
 ('sqft_living', 1814744.5822155084),
 ('sqft_lot', 218421.50915392386),
 ('floors', 4652.019754048737),
 ('waterfront', 583982.40628618),
 ('view', 201632.89693663333),
 ('condition', 127030.21214611766),
 ('grade', 962207.5939031859),
 ('sqft_above', 321107.94185402803),
 ('sqft_basement', 3.6955867985263517),
 ('yr_built', -289082.09092487907),
 ('yr_renovated', 46589.455113293625),
 ('lat', 348545.8847999194),
 ('long', -143547.71759468075),
 ('sqft_living15', 184163.69971163897),
 ('sqft_lot15', -343036.6459143963)]

### Evaluación del mejor modelo

A continuación realizaremos predicciones sobre el conjunto de pruebas para comparar con los valores de `y_test`. Utilizaremos la función `predict()` sobre el mejor modelo, y obtendremos tres métricas de desempeño:

In [25]:
y_pred = mejor_modelo.predict(x_test)

print('------ Modelo de regresión Lasso----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión Lasso----
RMSE: 200741.06
MAE: 126096.84
R²: 0.71


Con respecto al modelo de regresión lineal simple, el valor de R<sup>2</sup>, así como el valor de la raíz del error cuadrático medio y del error absoluto medio se mantienen similares. Es decir, para este conjunto de datos, el rendimiento de generalización no presenta mejorías con regularización L1 y un valor de `alpha` de 5.

## 6. Entrenamiento de un modelo de selección de variables

Si se entrena un modelo de regresión Lasso con un valor alto de `alpha`, la regularización puede llegar a anular los coeficientes de algunas variables. Este comportamiento permite realizar una selección de las variables menos importantes, anulando su influencia en las predicciones de la variable objetivo. Usando nuestro conjunto de datos, vamos a definir un objeto de tipo `Lasso()`, pero esta vez utilizando un valor de `alpha` de 500:

In [26]:
lasso_sv = Lasso(alpha=500)

Entrenaremos el modelo utilizando el conjunto de entrenamiento:

In [27]:
lasso_sv.fit(x_train, y_train)

Lasso(alpha=500, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

Y obtendremos los coeficientes resultantes:

In [28]:
list(zip(x_train.columns, lasso_sv.coef_))

[('bedrooms', -0.0),
 ('bathrooms', 256393.540379287),
 ('sqft_living', 1435840.0274614198),
 ('sqft_lot', 0.0),
 ('floors', 4567.45288353608),
 ('waterfront', 526246.1518650266),
 ('view', 227421.86840377992),
 ('condition', 102229.04443939257),
 ('grade', 1031926.5071658861),
 ('sqft_above', 311164.438861032),
 ('sqft_basement', 1048.1242579668674),
 ('yr_built', -274724.6461437568),
 ('yr_renovated', 44324.909241376525),
 ('lat', 349694.5890734177),
 ('long', -103478.41108580994),
 ('sqft_living15', 176003.15226584443),
 ('sqft_lot15', -0.0)]

Como puedes observar, el coeficiente de las variables `bedrooms`, `sqft_lot` y `sqft_lot15` es 0, lo que implica que este modelo no tiene en cuenta estas variables para realizar predicciones. Una de las particularidades de la regresión Lasso es que, gracias al uso del término de penalización, es capaz de determinar variables que no son relevantes para la estimación de la variable objetivo. Usualmente esta selección de variables se produce con valores grandes del hiperparámetro `alpha`, lo que implica que la regularización también es mucho mayor. Este resultado puede ser utilizado para, por ejemplo, establecer las variables más relevantes y entrenar un modelo diferente sobre el subconjunto de variables obtenido.